In [1]:
!nvidia-smi

Tue Mar  2 16:20:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Imports and installs

In [2]:
import numpy as np
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from sklearn.metrics import r2_score
import pickle
from pickle import dump
import re
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from nltk import WordNetLemmatizer
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize, sent_tokenize 
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse = False)
Lemmatizer = WordNetLemmatizer()
std_scaler = StandardScaler()
vectorizer = TfidfVectorizer(stop_words=None)

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
!pip install transformers

     |████████████████████████████████| 1.9MB 7.2MB/s 
     |████████████████████████████████| 890kB 33.4MB/s 
     |████████████████████████████████| 3.2MB 22.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=ef127fc0ce5e6326cbbf7695b09ba465b20cc2a676e5d4f7e0624590963e303c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


### IMPORT TRANSFORMERS

In [5]:
#from transformers import BertTokenizer, BertForTokenClassification, BertModel
#from transformers import AlbertTokenizer, AlbertModel
#from transformers import ElectraForTokenClassification, ElectraTokenizer
from transformers import RobertaForTokenClassification, RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base", add_prefix_space = True)
base_model = RobertaForTokenClassification.from_pretrained("roberta-base")

#tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
#base_model = BertForTokenClassification.from_pretrained("bert-base-uncased")
#base_model = BertModel.from_pretrained("bert-base-uncased")

#tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")
#base_model = ElectraForTokenClassification.from_pretrained("google/electra-small-discriminator")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able

# LOAD DATA

In [6]:
#train_data_file_path = "/content/drive/My Drive/CMCL Shared Task/preprocessed_dataset.csv"
#test_data_file_path = "/content/drive/My Drive/CMCL Shared Task/preprocessed_test_dataset.csv"
train_data_file_path = "/content/drive/My Drive/CMCL Shared Task/training_data.csv"
test_data_file_path = "/content/drive/My Drive/CMCL Shared Task/test_data.csv"

#file_path = "/content/drive/MyDrive/datasets/EyeGazeSharedTask/trial_data.csv"

train_data = pd.read_csv(train_data_file_path)
test_data = pd.read_csv(test_data_file_path)

In [7]:
print("Train Data Shape = ", train_data.shape)
print("Test Data Shape = ", test_data.shape)
df = train_data.copy()
test_df = test_data.copy()

train_data

Train Data Shape =  (15736, 8)
Test Data Shape =  (3554, 3)


,sentence_id,word_id,word,nFix,FFD,GPT,TRT,fixProp
0,0,0,Carlucci,28.397566,4.642973,6.190631,10.343594,94.117647
1,0,1,was,12.981744,3.534385,5.263977,4.565348,76.470588
2,0,2,deputy,25.152130,5.809781,9.926357,9.145251,100.000000
3,0,3,defense,20.283976,5.261551,8.092455,8.359293,88.235294
4,0,4,secretary,17.849899,4.135982,4.507129,5.686066,88.235294
...,...,...,...,...,...,...,...,...
15731,799,4,and,11.494253,3.326575,6.183489,3.807691,72.222222
15732,799,5,patient,23.754789,5.230420,7.826157,8.412660,100.000000
15733,799,6,wife,18.390805,3.674811,8.611980,6.669187,77.777778
15734,799,7,of,14.559387,4.380448,11.890443,6.114758,77.777778


In [8]:
test_data

,sentence_id,word_id,word
0,800,0,It's
1,800,1,the
2,800,2,funniest
3,800,3,American
4,800,4,comedy
...,...,...,...
3549,990,5,pursued
3550,990,6,a
3551,990,7,career
3552,990,8,in


In [9]:
df_data = pd.read_csv("/content/drive/My Drive/CMCL Shared Task/Sentences_maxlen.csv")
sentences = list(df_data["sentences"])
MAX_LEN = df_data["max_len"][0]

test_df_data = pd.read_csv("/content/drive/My Drive/CMCL Shared Task/Test_Sentences_maxlen.csv")
test_sentences = list(test_df_data["sentences"])
test_MAX_LEN = test_df_data["max_len"][0]

In [10]:
test_sentences[0]

"It's the funniest American comedy since Graffiti Bridge."

#### REMOVE EOS TOKEN

In [11]:
#@title
# FUNCTION TO REMOVE THE <EOS> TOKEN IN THE DATASET

def remove_eos(df, train = True):
  cnt = 1
  if not train:
    cnt = 801
  endword = []
  for i in range(df.shape[0]-1):
    if (df.loc[i+1, "sentence_id"] == cnt):
      df.loc[i, "word"] = df.loc[i, "word"][:-5]   # Remove <EOS> for the last word of each sentence.
      cnt += 1
      endword.append(1)
    else:
      endword.append(-1) 
  s = df.loc[df.shape[0] - 1, "word"]              # Remove <EOS> for last element separately
  df.loc[df.shape[0] - 1, "word"] = s[:-6] 
  endword.append(1)
  df["endword"] = endword
  return df

# Preprocessing

### COUNT NO OF TOKENS FOR EACH WORD

In [12]:
#@title
# FUNCTION TO CALCULATE NO OF TOKENS CREATED BY TOKENIZER PER WORD
# ADDS THE DATA IN A NEW COLUMN

def add_token_per_words(df):
  n_tokens_per_word = []
  cdf = 0
  cf_n_token_per_word = []
  for i, word in enumerate(df.word):
    n_tokens_per_word.append(len(tokenizer.encode(word)) - 2)
    if (i > 0) and (df.loc[i-1, "sentence_id"] != df.loc[i, "sentence_id"]):
      cdf = 0
    cdf += len(tokenizer.encode(word)) - 2
    cf_n_token_per_word.append(cdf)
  df["n_tokens"] = n_tokens_per_word
  df["cf_n_tokens"] = cf_n_token_per_word
  return df

add_token_per_words(df)
add_token_per_words(test_df)
df

,sentence_id,word_id,word,nFix,FFD,GPT,TRT,fixProp,n_tokens,cf_n_tokens
0,0,0,Carlucci,28.397566,4.642973,6.190631,10.343594,94.117647,3,3
1,0,1,was,12.981744,3.534385,5.263977,4.565348,76.470588,1,4
2,0,2,deputy,25.152130,5.809781,9.926357,9.145251,100.000000,1,5
3,0,3,defense,20.283976,5.261551,8.092455,8.359293,88.235294,1,6
4,0,4,secretary,17.849899,4.135982,4.507129,5.686066,88.235294,1,7
...,...,...,...,...,...,...,...,...,...,...
15731,799,4,and,11.494253,3.326575,6.183489,3.807691,72.222222,1,7
15732,799,5,patient,23.754789,5.230420,7.826157,8.412660,100.000000,1,8
15733,799,6,wife,18.390805,3.674811,8.611980,6.669187,77.777778,1,9
15734,799,7,of,14.559387,4.380448,11.890443,6.114758,77.777778,1,10


In [13]:
test_df

,sentence_id,word_id,word,n_tokens,cf_n_tokens
0,800,0,It's,2,2
1,800,1,the,1,3
2,800,2,funniest,2,5
3,800,3,American,1,6
4,800,4,comedy,1,7
...,...,...,...,...,...
3549,990,5,pursued,1,7
3550,990,6,a,1,8
3551,990,7,career,1,9
3552,990,8,in,1,10


### NO OF CHARS FOR EACH WORD

In [14]:
#@title
# FUNCTION TO CALCULATE THE NUMBER OF CHARACTERS PER WORD
# ADDS THE DATA IN A NEW COLUMN

def char_per_word(df):
  n_chars = []
  for word in df.word:
    n_chars.append(len(str(word)))
  df["n_chars"] = n_chars
  return df

### NO OF CHARS OF WORD - LEMMATIZED WORD

In [15]:
#@title
# FUNCTION TO CALCULATE THE DIFFERENCE BETWEEN NUMBER OF CHARACTERS IN WORD AND LEMMATIZED WORD
# ADDS AS A NEW COLUMN

def char_per_lemmatized_word(df):
  n_chars = []
  for word in df.word:
    n_chars.append(len(str(word)) - len(Lemmatizer.lemmatize(word)))
  df["n_char_lemmatized"] = n_chars
  return df

### STOP WORD OR NOT

In [16]:
#@title
import nltk
nltk.download("stopwords")
nltk.download("wordnet")

stopwords = nltk.corpus.stopwords
stop_words = stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [17]:
#@title
# FUNCTION TO ASSERT WHETHER A WORD IS STOPWORD  OR NOT
# ADDS THE DATA IN A NEW COLUMN

def add_stopword_check(df):
  if_stopword = []
  for word in df.word:
    if word in stop_words:
      if_stopword.append(1)
    else:
      if_stopword.append(-1)

  df["stopword"] = if_stopword
  return df

### NUMBER OR NOT

In [18]:
#@title
# FUNCTION TO DEFINE WHETHER IT IS A NUMBER OR NOT
# ADDS THE DATA AS A NEW COLUMN

def add_number_check(df):
  if_number = []
  for word in df.word:
    if_number.append(1 if word.isdigit() else -1)

  df["number"] = if_number
  return df

### TF IDF CALCULATION

In [19]:

# FUNCTION TO CALCULATE THE TFIDF OF THE TRAINING DATASET
# ADDS THE DATA IN A NEW COLUMN
# ALSO RETURNS A LIST OF THE SENTENCES

bad_words = []
punc = string.punctuation

def remove_punc(word):
  table = str.maketrans('', '', punc)
  return word.translate(table)

def calc_tfidf(df, train=True):
  start_pos = 0
  if not train:
    start_pos = 800
  n = np.array(df["sentence_id"])[-1]
  sentences = []
  tf_idfs = []
  MAX_LEN = 0
  for i in range(start_pos, n+1):   
    temp_df = df[df.sentence_id == i]
    sentence = (' ').join(np.array(temp_df.word))
    sentences.append(sentence)
    MAX_LEN = max(MAX_LEN, len(sentence.split()))
  if train:
    tf_idf = vectorizer.fit_transform(sentences)
  else:
    tf_idf = vectorizer.transform(sentences)

  for i, word in enumerate(df.word):
    try:
      tf_idfs.append(tf_idf.toarray()[df["sentence_id"][i]- start_pos][vectorizer.get_feature_names().index(remove_punc(word.lower()))])
    except:
      bad_words.append(word)
      if word in ["a", "A", "I"]:
        tf_idfs.append(0.8)
      else:
        tf_idfs.append(0.01)
  df["tf_idf"] = tf_idfs

  return sentences, df, bad_words, MAX_LEN

In [20]:

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

convert_dict = {"NNS":"NN",
                "PRP":"DT",
                "NN":"NN",
                "NNP":"NNP",
                "DT":"DT",
                "VB":"VB",
                "JJ":"JJ",
                "CD":"CD"}

def pos_tag_func(df, train=True):
  tags = []
  word_tag_tuple = nltk.pos_tag(df["word"])
  for word, tag in word_tag_tuple:
    if tag in convert_dict.keys():
      tags.append(convert_dict[tag]) 
    else:
      tags.append("unk")
  df["tags"] = tags
  
  if train:
    tag_transform = pd.DataFrame(enc.fit_transform(np.array(df.tags).reshape(-1, 1)))
  else:
    tag_transform = pd.DataFrame(enc.transform(np.array(df.tags).reshape(-1, 1)))
  df = pd.concat((df, tag_transform), axis = 1)
  return df

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [21]:

def use_transformed_GPT(df):
  df["GPT"] = df["TRT"] - df["GPT"]
  return df

In [22]:
# FUNCTION TO PERFORM ALL PREPROCESSING STEPS
def preprocess(df, train = True):
  return calc_tfidf(pos_tag_func(add_number_check(add_stopword_check(char_per_lemmatized_word(char_per_word(add_token_per_words(remove_eos(df, train)))))), train), train)


In [23]:
sentences, df, bad_words, MAX_LEN = preprocess(train_data)
test_sentences, test_df, test_bad_words, test_MAX_LEN = preprocess(test_data, False)
df1 = df.copy()
df1

,sentence_id,word_id,word,nFix,FFD,GPT,TRT,fixProp,endword,n_tokens,cf_n_tokens,n_chars,n_char_lemmatized,stopword,number,tags,0,1,2,3,4,5,6,tf_idf
0,0,0,Carlucci,28.397566,4.642973,6.190631,10.343594,94.117647,-1,3,3,8,0,-1,-1,NNP,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.178778
1,0,1,was,12.981744,3.534385,5.263977,4.565348,76.470588,-1,1,4,3,1,1,-1,unk,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.060409
2,0,2,deputy,25.152130,5.809781,9.926357,9.145251,100.000000,-1,1,5,6,0,-1,-1,JJ,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.178778
3,0,3,defense,20.283976,5.261551,8.092455,8.359293,88.235294,-1,1,6,7,0,-1,-1,NN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.310704
4,0,4,secretary,17.849899,4.135982,4.507129,5.686066,88.235294,-1,1,7,9,0,-1,-1,NN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.275261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15731,799,4,and,11.494253,3.326575,6.183489,3.807691,72.222222,-1,1,7,3,0,1,-1,unk,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.118080
15732,799,5,patient,23.754789,5.230420,7.826157,8.412660,100.000000,-1,1,8,7,0,-1,-1,JJ,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.454322
15733,799,6,wife,18.390805,3.674811,8.611980,6.669187,77.777778,-1,1,9,4,0,-1,-1,NN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.343320
15734,799,7,of,14.559387,4.380448,11.890443,6.114758,77.777778,-1,1,10,2,0,1,-1,unk,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.123557


In [24]:
train_data

,sentence_id,word_id,word,nFix,FFD,GPT,TRT,fixProp,endword,n_tokens,cf_n_tokens,n_chars,n_char_lemmatized,stopword,number,tags
0,0,0,Carlucci,28.397566,4.642973,6.190631,10.343594,94.117647,-1,3,3,8,0,-1,-1,NNP
1,0,1,was,12.981744,3.534385,5.263977,4.565348,76.470588,-1,1,4,3,1,1,-1,unk
2,0,2,deputy,25.152130,5.809781,9.926357,9.145251,100.000000,-1,1,5,6,0,-1,-1,JJ
3,0,3,defense,20.283976,5.261551,8.092455,8.359293,88.235294,-1,1,6,7,0,-1,-1,NN
4,0,4,secretary,17.849899,4.135982,4.507129,5.686066,88.235294,-1,1,7,9,0,-1,-1,NN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15731,799,4,and,11.494253,3.326575,6.183489,3.807691,72.222222,-1,1,7,3,0,1,-1,unk
15732,799,5,patient,23.754789,5.230420,7.826157,8.412660,100.000000,-1,1,8,7,0,-1,-1,JJ
15733,799,6,wife,18.390805,3.674811,8.611980,6.669187,77.777778,-1,1,9,4,0,-1,-1,NN
15734,799,7,of,14.559387,4.380448,11.890443,6.114758,77.777778,-1,1,10,2,0,1,-1,unk


In [25]:
train_data[train_data["word"]==""].count()

sentence_id          0
word_id              0
word                 0
nFix                 0
FFD                  0
GPT                  0
TRT                  0
fixProp              0
endword              0
n_tokens             0
cf_n_tokens          0
n_chars              0
n_char_lemmatized    0
stopword             0
number               0
tags                 0
dtype: int64

In [26]:
test_df

,sentence_id,word_id,word,endword,n_tokens,cf_n_tokens,n_chars,n_char_lemmatized,stopword,number,tags,0,1,2,3,4,5,6,tf_idf
0,800,0,It's,-1,2,2,4,0,-1,-1,NNP,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
1,800,1,the,-1,1,3,3,0,1,-1,DT,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.108989
2,800,2,funniest,-1,2,5,8,0,-1,-1,unk,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.519424
3,800,3,American,-1,1,6,8,0,-1,-1,JJ,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.296898
4,800,4,comedy,-1,1,7,6,0,-1,-1,NN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.356212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3549,990,5,pursued,-1,1,7,7,0,-1,-1,unk,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.010000
3550,990,6,a,-1,1,8,1,0,1,-1,DT,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.800000
3551,990,7,career,-1,1,9,6,0,-1,-1,NN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.398571
3552,990,8,in,-1,1,10,2,0,1,-1,unk,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.137170


In [27]:
# FUNCTION TO NORMALISE ALL THE TARGET VALUES
from sklearn.preprocessing import StandardScaler

training_dataset = pd.read_csv("/content/drive/My Drive/CMCL Shared Task/training_data.csv")
std_scaler = StandardScaler()
def standardize_target(df):
  df = use_transformed_GPT(df)
  keys = df.keys()[3:8]
  new_df = pd.DataFrame(std_scaler.fit_transform(df.iloc[:,3:8]), columns = keys)
  df.update(new_df)
  return 

standardize_target(training_dataset)
training_dataset

,sentence_id,word_id,word,nFix,FFD,GPT,TRT,fixProp
0,0,0,Carlucci,1.410757,1.020764,1.093189,1.382395,1.038428
1,0,1,was,-0.224947,0.241393,0.071056,-0.204384,0.361232
2,0,2,deputy,1.066398,1.841063,0.053680,1.053315,1.264160
3,0,3,defense,0.549860,1.455642,0.274461,0.837481,0.812696
4,0,4,secretary,0.291591,0.664334,0.466622,0.103379,0.812696
...,...,...,...,...,...,...,...,...
15731,799,4,and,-0.382778,0.095297,-0.282289,-0.412446,0.198203
15732,799,5,patient,0.918133,1.433756,0.341808,0.852136,1.264160
15733,799,6,wife,0.348984,0.340117,-0.191064,0.373357,0.411395
15734,799,7,of,-0.057551,0.836201,-0.998577,0.221104,0.411395


In [28]:
#standardize_target(df)

dump(std_scaler, open("/content/drive/My Drive/CMCL Shared Task/StandardScaler.pkl", "wb"))

In [29]:
df.to_csv("/content/drive/My Drive/CMCL Shared Task/preprocessed_dataset.csv", index = False)
test_df.to_csv("/content/drive/My Drive/CMCL Shared Task/preprocessed_test_dataset.csv", index = False)

In [30]:
sent_len_dict = {'sentences':sentences,
                 "max_len":MAX_LEN}
test_sent_len_dict = {'sentences':test_sentences,
                 "max_len":test_MAX_LEN}

pd.DataFrame(sent_len_dict).to_csv("/content/drive/My Drive/CMCL Shared Task/Sentences_maxlen.csv")
pd.DataFrame(test_sent_len_dict).to_csv("/content/drive/My Drive/CMCL Shared Task/Test_Sentences_maxlen.csv")

dump(std_scaler, open("/content/drive/My Drive/CMCL Shared Task/StandardScaler.pkl", "wb"))
dump(vectorizer, open("/content/drive/My Drive/CMCL Shared Task/TfIdfVectorizer.pkl", "wb"))
dump(enc, open("/content/drive/My Drive/CMCL Shared Task/OneHotEncoder.pkl", "wb"))

### FORM TARGETS

In [31]:
# FUNCTION TO FORM TARGETS AND OTHER FEATURES IN SEPARATE DATA STRUCTURES

def form_targets(sentences, df, MAX_LEN, train = True):
  start_pos = 800
  if train:
    start_pos = 0
  n = np.array(df["sentence_id"])[-1] - start_pos
  targets = []
  tags = []
  cdfs = []
  features = {"n_chars" : [],
              "stopword" : [],
              "number" : [],
              "endword":[],
              "n_char_lemmatized" : [],
              "tf_idf" : [],
              }

  for i in range(n+1):
    feature = {}
    actual_features = {}
    temp_df = df[df.sentence_id == (i+start_pos)]
    
    cdf = [0 for i in range(MAX_LEN - len(sentences[i].split()))]
    actual_cdf = list(np.array(temp_df.loc[:, "cf_n_tokens"]))
    actual_cdf += cdf

    if train:
      target = [[0] for i in range(MAX_LEN - len(sentences[i].split()))]
      actual_targets = [list(x) for x in np.array(temp_df.iloc[:, 5:6])]
      actual_targets += target

    for key in features.keys():
      feature[key] = [0 for j in range(MAX_LEN - len(sentences[i].split()))]
      actual_features[key] = list(np.array(temp_df.loc[:, key]))
      actual_features[key] += feature[key]

    tag = [[0 for j in range(6)] for k in range(MAX_LEN - len(sentences[i].split()))]
    actual_tag = [list(x) for x in np.array(temp_df.iloc[:, -8:-2])]
    actual_tag += tag

    for key in features.keys():
      features[key].append(actual_features[key]) 
    if train:
      targets.append(actual_targets) 
    tags.append(actual_tag)
    cdfs.append(actual_cdf)

  return targets, features, tags, cdfs


In [32]:
MAX_LEN = MAX_LEN + 3*10
targets, features, tags, cdfs = form_targets(sentences, df, MAX_LEN)
print("Target Shape = ", np.array(targets, dtype = "object").shape)
test_targets, test_features, test_tags, test_cdfs = form_targets(test_sentences, test_df, MAX_LEN, False)

Target Shape =  (800, 95, 1)


In [33]:
np.array(test_tags).shape

(191, 95, 6)

In [34]:

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=20, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss


# Data Loaders

In [35]:
# Class for the data
# Contains 4 keys -> Sentences, targets, input_ids(for BERT), attention_mask(for BERT) 

class EyeGaze_dataset(Dataset):

  def __init__(self, sentences, targets = None, cdfs = None, features = None, tags = None, tokenizer = None, train = True):
    self.sentences = sentences               # List of Sentences
    self.targets = targets                   # List of Padded Targets
    self.tokenizer = tokenizer               # Tokenizer to be used (BERT)
    self.cdfs = cdfs                         # Cumulative Frequency Function
    self.features = features
    self.tags = tags
    self.train = train
    
  def __len__(self):
    return len(self.sentences)               # No of Examples = 100

  def __getitem__(self, index):
    sentence = str(self.sentences[index])        # Get the review at the particular index
    if self.train:
      target = self.targets[index]                    # Get the target label at the particular index
    cdf = self.cdfs[index]
    feature = {}
    for key in features.keys():
      feature[key] = features[key][index]
    tag = tags[index]
    encoding = self.tokenizer.encode_plus(   # Encoder encoding the particular review
        sentence, 
        return_attention_mask = True,
        padding = "max_length",
        max_length = MAX_LEN,
        return_tensors = "pt"
    )
    # The class simply returns a dictionary of the following
    if self.train:
      return_dict = {"target":torch.tensor(target, dtype = torch.float32),
                    "input_ids":encoding["input_ids"].flatten(),
                    "attention_mask":encoding["attention_mask"].flatten(),
                    "cdf": torch.tensor(cdf, dtype = torch.int16),
                    "tag":torch.tensor(tag, dtype = torch.float32)}
    else:
      return_dict = {"input_ids":encoding["input_ids"].flatten(),
                    "attention_mask":encoding["attention_mask"].flatten(),
                    "cdf": torch.tensor(cdf, dtype = torch.int16),
                    "tag":torch.tensor(tag, dtype = torch.float32)}
    for key in features.keys():
      return_dict[key] = torch.tensor(feature[key], dtype = torch.float32)

    return return_dict

In [36]:
# Forms the DataLoaders with test_size = 0.2
# Each DataLoader has the Class dataset as their elements
def create_data_loader(sentences, targets, cdfs, features, tags, tokenizer, batch_size, train = True):
  data = EyeGaze_dataset(sentences, targets, cdfs, features, tags, tokenizer, train)
  
  if train:
    train, val = train_test_split(data, test_size = 0.2, shuffle = True)
    train_data_loader = DataLoader(train, batch_size = batch_size, shuffle = False)
    val_data_loader = DataLoader(val, batch_size = batch_size, shuffle = False)
    return train_data_loader, val_data_loader
  test_data_loader = DataLoader(data, batch_size = batch_size, shuffle = False)
  return test_data_loader
  

In [37]:
BATCH_SIZE = 4
train_data_loader, val_data_loader = create_data_loader(sentences, targets, cdfs, features, tags, tokenizer, BATCH_SIZE)
test_data_loader = create_data_loader(test_sentences, None, test_cdfs, test_features, test_tags, tokenizer, BATCH_SIZE, False)

In [38]:
temp = next(iter(train_data_loader))
print(temp.keys())
keys = ['target', 'input_ids', 'attention_mask',  'n_chars', 'n_char_lemmatized', 'cdf', 'stopword', 'number', 'tf_idf', "tag"]
for key in keys:
  print(temp[key].size())

dict_keys(['target', 'input_ids', 'attention_mask', 'cdf', 'tag', 'n_chars', 'stopword', 'number', 'endword', 'n_char_lemmatized', 'tf_idf'])
torch.Size([4, 95, 1])
torch.Size([4, 95])
torch.Size([4, 95])
torch.Size([4, 95])
torch.Size([4, 95])
torch.Size([4, 95])
torch.Size([4, 95])
torch.Size([4, 95])
torch.Size([4, 95])
torch.Size([4, 95, 6])


In [39]:
for d in test_data_loader:
  print("tags dims = ", d["tag"].size())

tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 95, 6])
tags dims =  torch.Size([4, 

In [40]:
base_model.config.output_hidden_states = True

In [41]:
class LanguageHeadLayer(torch.nn.Module):
  def __init__(self):
    super(LanguageHeadLayer, self).__init__()
    self.mid = torch.nn.Linear(in_features = base_model.config.hidden_size, out_features = 128)
    self.out = torch.nn.Linear(in_features = 128, out_features = 256)
    self.act1 = torch.nn.ReLU()
    self.act2 = torch.nn.GELU()
    self.drop = torch.nn.Dropout(p = 0.3)

  def forward(self, output):
    output = self.mid(output)
    output = self.act2(output)
    output = self.drop(output)
    output = self.out(output)

    return output
    
class ExtraFeatureModel(torch.nn.Module):
  def __init__(self):
    super(ExtraFeatureModel, self).__init__()
    self.inp = torch.nn.Linear(in_features = 12, out_features = 256)
    self.transformerlayer = torch.nn.TransformerEncoderLayer(d_model = 256, nhead = 8, dim_feedforward = 256, dropout = 0.2, activation = "relu")
    self.transformer = torch.nn.TransformerEncoder(self.transformerlayer, num_layers = 2)
    self.act1 = torch.nn.GELU()
    self.act2 = torch.nn.ReLU()

  def forward(self, features):
    output = self.inp(features)
    output = self.act1(output)
    output = self.transformerlayer(output)
    output = torch.squeeze(output, dim = 0)

    return output

In [42]:
class EyeGazeClassifier(torch.nn.Module):
  def __init__(self):
    super(EyeGazeClassifier, self).__init__()
    self.base_model = base_model
    self.languageheadlayer = LanguageHeadLayer()
    self.feature_model = ExtraFeatureModel()
    self.out = torch.nn.Linear(in_features = 256, out_features = 1)

  def map_predictions(self, outputs, attention_masks, cdfs):
    for i in range(outputs.size()[0]):    
      output = outputs[i]
      attention_mask = attention_masks[i]
      cdf = cdfs[i]

      attention_mask = torch.unsqueeze(attention_mask, dim = 1)

      output = output*attention_mask

      pred_tensor = torch.unsqueeze(torch.mean(output[1:cdf[0]+1], dim = 0), dim = 0)

      for j in range(0, len(cdf)-1):
        if (cdf[j+1] != 0):
          y = torch.unsqueeze(torch.mean(output[cdf[j]+1:cdf[j+1]+1], dim = 0), dim = 0)
          pred_tensor = torch.cat((pred_tensor, y), dim = 0)

      pred_pad = torch.tensor([[0 for a in range(base_model.config.hidden_size)] for k in range(MAX_LEN - pred_tensor.size()[0])], dtype = torch.float32).to(device)
      pred_tensor = torch.cat((pred_tensor, pred_pad), dim = 0)
      if i==0:
        pred_tensors = torch.unsqueeze(pred_tensor, dim = 0)
      else:
        pred_tensors = torch.cat((pred_tensors, torch.unsqueeze(pred_tensor, dim = 0)), dim = 0)

    return pred_tensors

  def forward(self, input_ids, attention_mask, cdf, extra_features):
    lang_output = self.base_model(input_ids = input_ids, attention_mask = attention_mask)
    lang_output = self.map_predictions(lang_output["hidden_states"][-1], attention_mask, cdf)
    lang_output = self.languageheadlayer(lang_output)
    feature_output = self.feature_model(extra_features)
    output = (lang_output+feature_output)/2
    output = self.out(output)
    
    return output


In [43]:
model = EyeGazeClassifier()
model = model.to(device)

# Util Funks

In [44]:
def target_initialise():
  targets = {}
  outputs = {}

  targets["r20"] = torch.tensor([], dtype = torch.float32)
  targets["r21"] = torch.tensor([], dtype = torch.float32)
  targets["r22"] = torch.tensor([], dtype = torch.float32)
  targets["r23"] = torch.tensor([], dtype = torch.float32)
  targets["r24"] = torch.tensor([], dtype = torch.float32)

  outputs["pred_r20"] = torch.tensor([], dtype = torch.float32)
  outputs["pred_r21"] = torch.tensor([], dtype = torch.float32)
  outputs["pred_r22"] = torch.tensor([], dtype = torch.float32)
  outputs["pred_r23"] = torch.tensor([], dtype = torch.float32)
  outputs["pred_r24"] = torch.tensor([], dtype = torch.float32)
  
  return targets, outputs

In [45]:
def predict_model(model, data_loader, device):
  model = model.eval()
  print("Calculating Predictions on Test Set...\n")
  outputs = []
  _, outputs = target_initialise()  
  final_outputs = []

  with torch.no_grad():
    for batch, d in tqdm(enumerate(data_loader), total=len(data_loader)):
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      cdf = d["cdf"].to(device)
      char_len = torch.unsqueeze(d["n_chars"], dim = 2).to(device)
      char_len_lemmatized = torch.unsqueeze(d["n_char_lemmatized"], dim = 2).to(device)
      if_stopword = torch.unsqueeze(d["stopword"], dim = 2).to(device)
      if_num = torch.unsqueeze(d['number'], dim = 2).to(device)
      if_end = torch.unsqueeze(d["endword"], dim = 2).to(device)
      tfidf = torch.unsqueeze(d["tf_idf"], dim = 2)  .to(device) 
      tag = d["tag"].to(device)
      extra_feature = torch.cat((char_len, char_len_lemmatized, if_stopword, if_num, if_end, tfidf, tag), dim = 2).to(device)
      
      output = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        cdf = cdf,
        extra_features = extra_feature
        )
      
      for i in range(output.size()[0]):
        seq_len = len(test_sentences[i+batch*BATCH_SIZE].split())
        final_outputs += output[i, : seq_len, : ].tolist()

    
  return final_outputs

# Preds df

In [46]:
preds = pd.DataFrame()

In [124]:
col = "FFD"

# LOAD MODEL AND PREDICT

In [128]:
save_file_path = "/content/drive/My Drive/CMCL Shared Task/single target training/{}_RobertaToken_single_target.pth".format('ffd')
model = EyeGazeClassifier()
model.load_state_dict(torch.load(save_file_path, map_location = torch.device("cpu")))
model.eval()
""

''

In [129]:
device='cpu'

In [130]:
predictions = predict_model(model.to(device), test_data_loader, device)

Calculating Predictions on Test Set...



In [131]:
test_df

,sentence_id,word_id,word,endword,n_tokens,cf_n_tokens,n_chars,n_char_lemmatized,stopword,number,tags,0,1,2,3,4,5,6,tf_idf
0,800,0,It's,-1,2,2,4,0,-1,-1,NNP,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
1,800,1,the,-1,1,3,3,0,1,-1,DT,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.108989
2,800,2,funniest,-1,2,5,8,0,-1,-1,unk,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.519424
3,800,3,American,-1,1,6,8,0,-1,-1,JJ,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.296898
4,800,4,comedy,-1,1,7,6,0,-1,-1,NN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.356212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3549,990,5,pursued,-1,1,7,7,0,-1,-1,unk,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.010000
3550,990,6,a,-1,1,8,1,0,1,-1,DT,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.800000
3551,990,7,career,-1,1,9,6,0,-1,-1,NN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.398571
3552,990,8,in,-1,1,10,2,0,1,-1,unk,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.137170


In [132]:
print(test_df.shape)
np.array(predictions).shape

(3554, 19)


(3554, 1)

In [133]:
predictions = np.array(predictions).squeeze()

In [134]:
preds[col] = predictions
preds

,nFix,fixProp,GPT,TRT,FFD
0,7.849546,23.396795,4.006784,3.635208,2.525077
1,12.612781,56.713287,3.337132,4.224606,3.032670
2,21.752722,91.492058,7.173223,8.214662,4.372114
3,18.006620,93.937263,7.893115,8.012938,4.367565
4,16.986723,92.585632,7.421673,9.047362,4.187050
...,...,...,...,...,...
3549,28.412594,95.287804,7.941898,10.061141,4.876679
3550,8.420675,31.636440,3.814031,3.879652,3.370640
3551,17.659592,84.088860,8.287592,5.287661,3.739113
3552,7.001290,36.370205,4.568286,3.366920,2.550157


In [135]:
training = pd.read_csv("/content/drive/My Drive/CMCL Shared Task/training_data.csv")
training

,sentence_id,word_id,word,nFix,FFD,GPT,TRT,fixProp
0,0,0,Carlucci,28.397566,4.642973,6.190631,10.343594,94.117647
1,0,1,was,12.981744,3.534385,5.263977,4.565348,76.470588
2,0,2,deputy,25.152130,5.809781,9.926357,9.145251,100.000000
3,0,3,defense,20.283976,5.261551,8.092455,8.359293,88.235294
4,0,4,secretary,17.849899,4.135982,4.507129,5.686066,88.235294
...,...,...,...,...,...,...,...,...
15731,799,4,and,11.494253,3.326575,6.183489,3.807691,72.222222
15732,799,5,patient,23.754789,5.230420,7.826157,8.412660,100.000000
15733,799,6,wife,18.390805,3.674811,8.611980,6.669187,77.777778
15734,799,7,of,14.559387,4.380448,11.890443,6.114758,77.777778


In [136]:
preds.min(axis=0)

nFix       0.169233
fixProp    2.158408
GPT        0.010068
TRT        0.000402
FFD       -1.091538
dtype: float64

In [137]:
preds.median(axis=0)

nFix       13.458487
fixProp    67.441498
GPT         4.899055
TRT         4.611491
FFD         2.912300
dtype: float64

In [138]:
training.median(axis=0)[col] 

3.3231382521736137

In [139]:
actual_median = training.median(axis=0)[col] 

In [140]:
preds[col][preds[col]<0]

201    -0.077444
205    -0.028611
208    -0.131433
307    -0.066306
308    -0.119467
403    -0.005607
444    -0.080230
445    -0.075738
447    -0.131123
475    -0.501463
578    -0.178337
594    -0.022189
599    -0.674830
842    -0.264447
993    -0.181708
1268   -0.016729
1553   -0.047900
1592   -0.044369
1693   -0.215312
1843   -0.004885
1846   -0.129218
2160   -0.092558
2173   -1.091538
2174   -0.248763
2307   -0.350603
2317   -0.038867
2339   -0.061955
2384   -0.052949
2437   -0.467388
2528   -0.036574
2561   -0.161408
2670   -0.048799
2740   -0.065631
2996   -0.257096
3098   -0.003341
3289   -0.190792
3330   -0.011992
3361   -0.887687
3386   -0.210757
Name: FFD, dtype: float64

In [141]:
preds[col][preds[col]<0] = actual_median

In [142]:
preds.min(axis=0)

nFix       0.169233
fixProp    2.158408
GPT        0.010068
TRT        0.000402
FFD        0.002907
dtype: float64

In [143]:
preds

,nFix,fixProp,GPT,TRT,FFD
0,7.849546,23.396795,4.006784,3.635208,2.525077
1,12.612781,56.713287,3.337132,4.224606,3.032670
2,21.752722,91.492058,7.173223,8.214662,4.372114
3,18.006620,93.937263,7.893115,8.012938,4.367565
4,16.986723,92.585632,7.421673,9.047362,4.187050
...,...,...,...,...,...
3549,28.412594,95.287804,7.941898,10.061141,4.876679
3550,8.420675,31.636440,3.814031,3.879652,3.370640
3551,17.659592,84.088860,8.287592,5.287661,3.739113
3552,7.001290,36.370205,4.568286,3.366920,2.550157


In [144]:
del model

# Combine Preds

In [145]:
preds

,nFix,fixProp,GPT,TRT,FFD
0,7.849546,23.396795,4.006784,3.635208,2.525077
1,12.612781,56.713287,3.337132,4.224606,3.032670
2,21.752722,91.492058,7.173223,8.214662,4.372114
3,18.006620,93.937263,7.893115,8.012938,4.367565
4,16.986723,92.585632,7.421673,9.047362,4.187050
...,...,...,...,...,...
3549,28.412594,95.287804,7.941898,10.061141,4.876679
3550,8.420675,31.636440,3.814031,3.879652,3.370640
3551,17.659592,84.088860,8.287592,5.287661,3.739113
3552,7.001290,36.370205,4.568286,3.366920,2.550157


In [146]:
preds = preds[['nFix', 'FFD', 'GPT', 'TRT', 'fixProp']] #nFix,FFD,GPT,TRT,fixProp
preds

,nFix,FFD,GPT,TRT,fixProp
0,7.849546,2.525077,4.006784,3.635208,23.396795
1,12.612781,3.032670,3.337132,4.224606,56.713287
2,21.752722,4.372114,7.173223,8.214662,91.492058
3,18.006620,4.367565,7.893115,8.012938,93.937263
4,16.986723,4.187050,7.421673,9.047362,92.585632
...,...,...,...,...,...
3549,28.412594,4.876679,7.941898,10.061141,95.287804
3550,8.420675,3.370640,3.814031,3.879652,31.636440
3551,17.659592,3.739113,8.287592,5.287661,84.088860
3552,7.001290,2.550157,4.568286,3.366920,36.370205


In [147]:
test_data = pd.read_csv('/content/drive/My Drive/CMCL Shared Task/test_data.csv')
test_data

,sentence_id,word_id,word
0,800,0,It's
1,800,1,the
2,800,2,funniest
3,800,3,American
4,800,4,comedy
...,...,...,...
3549,990,5,pursued
3550,990,6,a
3551,990,7,career
3552,990,8,in


In [148]:
submission = pd.concat([test_data, preds], axis = 1)
submission

,sentence_id,word_id,word,nFix,FFD,GPT,TRT,fixProp
0,800,0,It's,7.849546,2.525077,4.006784,3.635208,23.396795
1,800,1,the,12.612781,3.032670,3.337132,4.224606,56.713287
2,800,2,funniest,21.752722,4.372114,7.173223,8.214662,91.492058
3,800,3,American,18.006620,4.367565,7.893115,8.012938,93.937263
4,800,4,comedy,16.986723,4.187050,7.421673,9.047362,92.585632
...,...,...,...,...,...,...,...,...
3549,990,5,pursued,28.412594,4.876679,7.941898,10.061141,95.287804
3550,990,6,a,8.420675,3.370640,3.814031,3.879652,31.636440
3551,990,7,career,17.659592,3.739113,8.287592,5.287661,84.088860
3552,990,8,in,7.001290,2.550157,4.568286,3.366920,36.370205


In [149]:
submission.to_csv('/content/drive/My Drive/CMCL Shared Task/single target training/roberta_std_scaler_submission_v2.txt', index=False)